Loading the dataset csv's into data frames and printing the first 10 values in the data set for each data frame to ensure that the data is being read and stored correctly

In [34]:
import pandas as pd

dataset_1 = pd.read_csv("CO2 emission by countries.csv")
dataset_2 = pd.read_csv("Historical Tropical Storm.csv")
dataset_3 = pd.read_csv("global_temps.csv")

print("From Dataset 1: CO2 emission by countries")
print(dataset_1.head(10))
print("\n")

print("From Dataset 2: Historical Tropical Storm")
print(dataset_2.head(10))
print("\n")

print("From Dataset 3: global_temps")
print(dataset_3.head(10))
print("\n")

From Dataset 1: CO2 emission by countries
       Country  Year  CO2 emission (Tons)
0  Afghanistan  1750                  0.0
1  Afghanistan  1751                  0.0
2  Afghanistan  1752                  0.0
3  Afghanistan  1753                  0.0
4  Afghanistan  1754                  0.0
5  Afghanistan  1755                  0.0
6  Afghanistan  1756                  0.0
7  Afghanistan  1757                  0.0
8  Afghanistan  1758                  0.0
9  Afghanistan  1759                  0.0


From Dataset 2: Historical Tropical Storm
   YEAR  MONTH  DAY   LAT  LONG  WIND_KTS  PRESSURE CAT  Shape_Leng Country
0  1851      7    5  22.2 -97.6        80         0  H1    0.141421  Mexico
1  1851      8   21  23.2 -82.5        60         0  TS    1.303840    Cuba
2  1851      8   21  23.9 -83.6        70         0  H1    0.860233    Cuba
3  1851      8   20  21.2 -79.0        70         0  H1    1.565248    Cuba
4  1851      8   20  21.9 -80.4        70         0  H1    1.220656    C

The "CO2 emission by countries" dataset currently has these attributes: "Code", "Calling Code", "Year", "CO2 emission (Tons)", "Population(2022)", "Area", "% of World" and "Density(km2)" but from this dataset we only require "Country", "Year" and "CO2 emission (Tons)" so the others will be removed to reduce the size of the final dataset

In [35]:
#Removing unnecessary columns from the datasets "CO2 emission by countries"
dataset_1.columns = dataset_1.columns.str.strip()
dropping = ["Code", "Calling Code", "Population(2022)", "Area", "% of World", "Density(km2)"]
dataset_1.drop(columns=dropping, inplace= True, errors= "ignore")

#updating the csv
dataset_1.to_csv("CO2 emission by countries.csv", index= False)
print("CO2 emission by countries dataset, has been updated")

CO2 emission by countries dataset, has been updated


We will now use the Geopy module to deduce the countries where the Historical Tropical storms occured based on the Latitude and Longititude provided. This will be of use later on

In [36]:
from geopy.geocoders import Nominatim
import reverse_geocoder as rg
import pycountry
# !pip install reverse_geocoder
geolocator = Nominatim(user_agent="my_geopy_app")



Function to extract country name from pycountry response

In [37]:
def get_country_name(code):
    try:
        return pycountry.countries.get(alpha_2=code).name
    except:
        return 'Unknown'

In [38]:
coords = list(zip(dataset_2['LAT'], dataset_2['LONG']))

# Run reverse geocoding in bulk
results = rg.search(coords)  # returns list of dicts with keys like 'cc' (country code), 'name', etc.

# Extract country info
country_names = [res['cc'] for res in results]  # you can also use res['name'] or 'admin1' if needed

country_full_names = [get_country_name(code) for code in country_names]

# Add to DataFrame
dataset_2["Country"] = country_full_names

# Save result
# dataset_2.to_csv("Historical Tropical Storm-v2.csv", index=False)


The "Historical Tropical Storm" dataset currently has these attributes: "FID", "YEAR", "MONTH", "DAY", "AD_TIME", "BTID", "NAME", "LAT", "LONG", "WIND_KTS", "PRESSURE", "CAT", "BASIN" and "Shape_Leng" but from this dataset we only require "YEAR", "MONTH", "DAY", "LAT", "LONG", "WIND_KTS", "PRESSURE", "CAT" and "Shape_Leng" so the others will be removed to reduce the size of the final dataset

In [39]:
#Removing unnecessary columns from the dataset "Historical Tropical Storm"
dataset_2.columns = dataset_2.columns.str.strip()
dropping_2 = ["FID", "AD_TIME", "BTID", "NAME", "BASIN"]
dataset_2.drop(columns=dropping_2, inplace= True, errors= "ignore")

#updating the csv
dataset_2.to_csv("Historical Tropical Storm.csv", index= False)
print("Historical Tropical Storm dataset, has been updated")

Historical Tropical Storm dataset, has been updated


The "global_temps" dataset currently has these attributes: "Year", "Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec", "J-D", "D-N", "DJF", "MAM", "JJA" and "SON" but from this dataset we only require "Year", "Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov"and "Dec" so the others will be removed to reduce the size of the final dataset

In [40]:
#Removing unnecessary columns from the dataset "global_temps"
dataset_3.columns = dataset_3.columns.str.strip()
dropping_3 = ["J-D", "D-N", "DJF", "MAM", "JJA", "SON"]
dataset_3.drop(columns=dropping_3, inplace= True, errors= "ignore")

#updating the csv
dataset_3.to_csv("global_temps.csv", index= False)
print("global_temps dataset, has been updated")

global_temps dataset, has been updated


Cleaning the data by replacing nan values with approprate values for the specific column, by first identifying where the nan values are in the datasets

In [41]:
#Identify the number of missing values in the datasets
print("Missing values from Dataset 1: CO2 emission by countries")
print(dataset_1.isnull().sum())
print("\n")

print("Missing values from Dataset 2: Historical Tropical Storm")
print(dataset_2.isnull().sum())
print("\n")

print("Missing values from Dataset 3: global_temps")
print(dataset_3.isnull().sum())
print("\n")

Missing values from Dataset 1: CO2 emission by countries
Country                0
Year                   0
CO2 emission (Tons)    0
dtype: int64


Missing values from Dataset 2: Historical Tropical Storm
YEAR          0
MONTH         0
DAY           0
LAT           0
LONG          0
WIND_KTS      0
PRESSURE      0
CAT           0
Shape_Leng    0
Country       0
dtype: int64


Missing values from Dataset 3: global_temps
Year    0
Jan     0
Feb     0
Mar     0
Apr     0
May     0
Jun     0
Jul     0
Aug     0
Sep     0
Oct     0
Nov     0
Dec     0
dtype: int64




In [42]:
#Cleaning the global_temps dataset

#Filling in all nan values with the avarage value of their column
find_avg_value= ["Jun","Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

#temp assigning the nan values to 0 for the summing
dataset_3[find_avg_value] = dataset_3[find_avg_value].fillna(0)

#casting to float
dataset_3[find_avg_value] = dataset_3[find_avg_value].astype(float)

agv_value = dataset_3[find_avg_value].mean()
dataset_3[find_avg_value] = dataset_3[find_avg_value].fillna(agv_value) 


#Updating the dataset_3 csv
dataset_3.to_csv("global_temps.csv", index= False)

#Testing if the change was made
print("Missing values from Dataset 3: global_temps")
print(dataset_3.isnull().sum())
print("\n")


Missing values from Dataset 3: global_temps
Year    0
Jan     0
Feb     0
Mar     0
Apr     0
May     0
Jun     0
Jul     0
Aug     0
Sep     0
Oct     0
Nov     0
Dec     0
dtype: int64




Checking if the current data types for all attributes in all datasets are of the correct types for further processing. To further reduce the size of the final dataset I will be down casting the datatypes

In [43]:
#dataset_1

#Before
print("Before") 
dataset_1.info()
print("\n")

#Down casting
dataset_1["Year"] = dataset_1["Year"].astype("int32")
dataset_1["CO2 emission (Tons)"] = dataset_1["CO2 emission (Tons)"].astype("float32")

#Updating the dataset_1 csv
dataset_1.to_csv("CO2 emission by countries.csv", index= False)

#After
print("After") 
dataset_1.info()

Before
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59620 entries, 0 to 59619
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Country              59620 non-null  object 
 1   Year                 59620 non-null  int64  
 2   CO2 emission (Tons)  59620 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.4+ MB


After
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59620 entries, 0 to 59619
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Country              59620 non-null  object 
 1   Year                 59620 non-null  int32  
 2   CO2 emission (Tons)  59620 non-null  float32
dtypes: float32(1), int32(1), object(1)
memory usage: 931.7+ KB


In [44]:
#dataset_2

#Before 
print("Before") 
dataset_2.info()
print("\n")

#Down casting
to_int_32 = ["YEAR", "MONTH", "DAY", "WIND_KTS", "PRESSURE"]
dataset_2[to_int_32 ] = dataset_2[to_int_32 ].astype("int32")

to_float_32 = ["LAT", "LONG", "Shape_Leng"]
dataset_2[to_float_32] = dataset_2[to_float_32].astype("float32")

#Updating the dataset_1 csv
dataset_2.to_csv("Historical Tropical Storm.csv", index= False)

#After
print("After") 
dataset_2.info()


Before
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59228 entries, 0 to 59227
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   YEAR        59228 non-null  int64  
 1   MONTH       59228 non-null  int64  
 2   DAY         59228 non-null  int64  
 3   LAT         59228 non-null  float64
 4   LONG        59228 non-null  float64
 5   WIND_KTS    59228 non-null  int64  
 6   PRESSURE    59228 non-null  int64  
 7   CAT         59228 non-null  object 
 8   Shape_Leng  59228 non-null  float64
 9   Country     59228 non-null  object 
dtypes: float64(3), int64(5), object(2)
memory usage: 4.5+ MB


After
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59228 entries, 0 to 59227
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   YEAR        59228 non-null  int32  
 1   MONTH       59228 non-null  int32  
 2   DAY         59228 non-null  int32  
 3   LAT

In [45]:
#dataset_3

#Before 
print("Before") 
dataset_3.info()
print("\n")

#Down casting
dataset_3["Year"] = dataset_3["Year"].astype("int32")

to_float_32_d3 = ["Jan", "Feb", "Mar", "Apr", "May", "Jun","Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
dataset_3[to_float_32_d3] = dataset_3[to_float_32_d3].astype("float32")

#Updating the dataset_1 csv
dataset_3.to_csv("global_temps.csv", index= False)

#After
print("After") 
dataset_3.info()

Before
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Year    144 non-null    int64  
 1   Jan     144 non-null    float64
 2   Feb     144 non-null    float64
 3   Mar     144 non-null    float64
 4   Apr     144 non-null    float64
 5   May     144 non-null    float64
 6   Jun     144 non-null    float64
 7   Jul     144 non-null    float64
 8   Aug     144 non-null    float64
 9   Sep     144 non-null    float64
 10  Oct     144 non-null    float64
 11  Nov     144 non-null    float64
 12  Dec     144 non-null    float64
dtypes: float64(12), int64(1)
memory usage: 14.8 KB


After
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Year    144 non-null    int32  
 1   Jan     144 non-null    float32
 2   Feb     144 non

Merging the 3 datasets together, dataset_2 "Historical Tropical Storm" with joining dataset_1 "CO2 emission by countries" using their year attribute, then taking the joined datasets and connecting it to "global_temps" using the year attribute 

In [46]:
#import csv

#changing the attribute names to match where the are being joined
dataset_2 = dataset_2.rename(columns={"YEAR": "Year"})

dataset_4 = dataset_2.merge(dataset_1, on= ["Year","Country"], how= "left") #left for keeping all data from dataset_1, now merges based on the country name and year
# dataset_4 = pd.merge(dataset_1, dataset_2, on= ["Year","Country"])

print("Merged Dataset of Historical Tropical Storm and CO2 emission by countries")
print(dataset_4.head(5))

final_dataset = dataset_4.merge(dataset_3, on= "Year", how= "left") #left for keeping all data from dataset_4

print("Sample of the Final Dataset")
print(final_dataset.head(5))

#creating and writing to a new csv
final_dataset.to_csv("completed_dataset_for_IS_project_25.csv", index= False, encoding= "utf-8")

print("Final Dataset csv has been created")

Merged Dataset of Historical Tropical Storm and CO2 emission by countries
   Year  MONTH  DAY        LAT       LONG  WIND_KTS  PRESSURE CAT  Shape_Leng  \
0  1851      7    5  22.200001 -97.599998        80         0  H1    0.141421   
1  1851      8   21  23.200001 -82.500000        60         0  TS    1.303840   
2  1851      8   21  23.900000 -83.599998        70         0  H1    0.860233   
3  1851      8   20  21.200001 -79.000000        70         0  H1    1.565248   
4  1851      8   20  21.900000 -80.400002        70         0  H1    1.220656   

  Country  CO2 emission (Tons)  
0  Mexico                  0.0  
1    Cuba                  0.0  
2    Cuba                  0.0  
3    Cuba                  0.0  
4    Cuba                  0.0  
Sample of the Final Dataset
   Year  MONTH  DAY        LAT       LONG  WIND_KTS  PRESSURE CAT  Shape_Leng  \
0  1851      7    5  22.200001 -97.599998        80         0  H1    0.141421   
1  1851      8   21  23.200001 -82.500000        60

The merged data set currently has 1,048,576 rows, I'm going to remove te rows that lacks the necessary information further reducing the size of the dataset

The necessary columns are:  "Year", "MONTH", "DAY", "LAT", "LONG", "WIND_KTS", "PRESSURE", "CAT", "Shape_Leng", "CO2 emission (Tons)", "Jan", "Feb", "Mar", "Apr", "May", "Jun","Jul", "Aug", "Sep", "Oct", "Nov" and "Dec"

If any data is missing then remove that row

In [47]:
import dask.dataframe as dd

necessary_columns = ["Year", "MONTH", "DAY", "LAT", "LONG", "WIND_KTS", "PRESSURE", "CAT", "Shape_Leng", "Country", "CO2 emission (Tons)", "Jan", "Feb", "Mar", "Apr", "May", "Jun","Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

#made more efficient and reduces the amount of memory used
large_dataset = dd.read_csv("completed_dataset_for_IS_project_25.csv", usecols=necessary_columns)

cleaned_final_dataset = large_dataset.dropna(subset=necessary_columns)

print("Sample of the final dataset")
print(cleaned_final_dataset.head(5))

#Updating the dataset_3 csv - made more efficient
cleaned_final_dataset.compute().to_csv("completed_dataset_for_IS_project_25.csv", index= False)

print("The dataset is ready for processing")


Sample of the final dataset


      Year  MONTH  DAY   LAT  LONG  WIND_KTS  PRESSURE CAT  Shape_Leng  \
3520  1880      8   11  23.0 -91.9        70         0  H1    0.806226   
3521  1880      8   11  23.4 -92.6        80         0  H1    0.761577   
3522  1880      8   11  23.7 -93.3        80         0  H1    0.583095   
3523  1880      8   12  24.0 -93.8        90         0  H2    0.670820   
3524  1880      9    6  23.9 -88.6        40         0  TS    0.360555   

     Country  ...   Mar   Apr  May   Jun   Jul   Aug   Sep   Oct   Nov   Dec  
3520  Mexico  ... -0.09 -0.17 -0.1 -0.21 -0.18 -0.11 -0.15 -0.24 -0.22 -0.18  
3521  Mexico  ... -0.09 -0.17 -0.1 -0.21 -0.18 -0.11 -0.15 -0.24 -0.22 -0.18  
3522  Mexico  ... -0.09 -0.17 -0.1 -0.21 -0.18 -0.11 -0.15 -0.24 -0.22 -0.18  
3523  Mexico  ... -0.09 -0.17 -0.1 -0.21 -0.18 -0.11 -0.15 -0.24 -0.22 -0.18  
3524  Mexico  ... -0.09 -0.17 -0.1 -0.21 -0.18 -0.11 -0.15 -0.24 -0.22 -0.18  

[5 rows x 23 columns]
The dataset is ready for processing


In [48]:
#Testing to ensure no nan values in the dataset
print("Missing values from Final Dataset")
print(cleaned_final_dataset.isna().sum().compute())
print("\n")


Missing values from Final Dataset
Year                   0
MONTH                  0
DAY                    0
LAT                    0
LONG                   0
WIND_KTS               0
PRESSURE               0
CAT                    0
Shape_Leng             0
Country                0
CO2 emission (Tons)    0
Jan                    0
Feb                    0
Mar                    0
Apr                    0
May                    0
Jun                    0
Jul                    0
Aug                    0
Sep                    0
Oct                    0
Nov                    0
Dec                    0
dtype: int64




Down casing the final dataset

In [49]:
#Before 
cleaned_final_dataset.info()

#Down casting
to_int_32_final = ["Year", "MONTH", "DAY", "WIND_KTS", "PRESSURE"]
cleaned_final_dataset[to_int_32_final] = cleaned_final_dataset[to_int_32_final ].astype("int32")

to_float_32_final = ["CO2 emission (Tons)", "LAT", "LONG", "Shape_Leng", "Jan", "Feb", "Mar", "Apr", "May", "Jun","Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
cleaned_final_dataset[to_float_32_final] = cleaned_final_dataset[to_float_32_final].astype("float32")

#Updating the cleaned_final_dataset csv
cleaned_final_dataset.compute().to_csv("completed_dataset_for_IS_project_25.csv", index= False)

#After
cleaned_final_dataset.info()

<class 'dask.dataframe.dask_expr.DataFrame'>
Columns: 23 entries, Year to Dec
dtypes: float64(16), int64(5), string(2)<class 'dask.dataframe.dask_expr.DataFrame'>
Columns: 23 entries, Year to Dec
dtypes: float32(16), int32(5), string(2)

In [50]:
data = pd.read_csv("completed_dataset_for_IS_project_25.csv")

print("From Data: Completed dataset")
print(data.head(10))
print("\n")

From Data: Completed dataset
   Year  MONTH  DAY   LAT  LONG  WIND_KTS  PRESSURE CAT  Shape_Leng Country  \
0  1880      8   11  23.0 -91.9        70         0  H1    0.806226  Mexico   
1  1880      8   11  23.4 -92.6        80         0  H1    0.761577  Mexico   
2  1880      8   11  23.7 -93.3        80         0  H1    0.583095  Mexico   
3  1880      8   12  24.0 -93.8        90         0  H2    0.670820  Mexico   
4  1880      9    6  23.9 -88.6        40         0  TS    0.360555  Mexico   
5  1880      8   10  22.0 -89.6        60         0  TS    0.854400  Mexico   
6  1880      8    9  21.0 -87.4        70         0  H1    0.761577  Mexico   
7  1880      8   10  21.3 -88.1        60         0  TS    0.854400  Mexico   
8  1880      8   10  21.6 -88.9        50         0  TS    0.806226  Mexico   
9  1880     10    6  21.4 -87.3        50         0  TS    0.800000  Mexico   

   ...   Mar   Apr  May   Jun   Jul   Aug   Sep   Oct   Nov   Dec  
0  ... -0.09 -0.17 -0.1 -0.21 -0.

In [51]:
data.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
52651    False
52652    False
52653    False
52654    False
52655    False
Length: 52656, dtype: bool

For finding how strong a storm was, the higher the value the stronger the storm

In [ ]:
storm_intensity = []
for index, row in data.iterrows():
    stormIntensity = row["WIND_KTS"] * row["Shape_Leng"]
    storm_intensity.append(stormIntensity)
data["Sorm Intensity"] = storm_intensity
# data.to_csv("stom int.csv",index=False)

In [53]:
print(data['CAT'].unique())

['H1' 'H2' 'TS' 'E' 'H3' 'H4' 'TD' 'H5' 'L' 'SS' 'SD' 'W']


In [ ]:
# def categorize_storm_variable(abbreviation):
#     if abbreviation == 'H1':
#         return 'Hurricane Categroy 1'
#     elif abbreviation == 'H2':
#         return 'Hurricane Categroy 2'
#     elif abbreviation == 'H3':
#         return 'Hurricane Categroy 3'
#     elif abbreviation == 'H4':
#         return 'Hurricane Categroy 4'
#     elif abbreviation == 'H5':
#         return 'Hurricane Categroy 5'
#     elif abbreviation == 'TS':
#         return 'Tropical Storm'
#     elif abbreviation == 'E':
#         return 'Extra-tropical cyclone'
#     elif abbreviation == 'W':
#         return 'Warning'
#     elif abbreviation == 'L':
#         return 'Low-pressure system'
#     elif abbreviation == 'SS':
#         return 'Severe Storm'
#     elif abbreviation == 'SD':
#         return 'Subtropical Depression'
#     elif abbreviation == 'TD':
#         return 'Tropical Depression'
#     else:
#         return 'Unknown'

In [54]:
intensity_mapping = {
    'L': 0,    # Low-pressure system
    'TD': 1,   # Tropical Depression
    'SD': 1,   # Subtropical Depression (or possibly 0.5)
    'TS': 2,   # Tropical Storm
    'SS': 2,   # Severe Storm (if equivalent to TS)
    'H1': 3,   # Hurricane Category 1
    'H2': 4,
    'H3': 5,
    'H4': 6,
    'H5': 7,
    'E': -1,   # Extra-tropical (non-standard storm)
    'W': -2    # Warning flag, not intensity
}

For labelling each category of storm numerically for modelling

In [ ]:
data['Storm Intensity Label'] = data['CAT'].map(intensity_mapping)

# data.to_csv("types of storms.csv", index=False)

For measuring the wind speed intensity

In [ ]:
data['Wind Speed Squared'] = data['WIND_KTS'] ** 2

# data.to_csv("wind squared.csv", index = False)
